In [3]:
using Yao
using Yao.ConstGate # needed for P1 = 0.5*(I - sigma_z) block
using FreqTables
using LightGraphs
using Compose
using PyCall

In [4]:
#!/usr/bin/env julia


#=
H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j
=#

const u = 1.35
const Ω_max = 1.89
const δ_0 = -1.0
const δ_max = 1.0

function get_edges(graph::Vector{NTuple{2, Float64}})
    Nv = size(graph)[1]
    edges = falses(Nv, Nv)
    for i in 1:(Nv-1)
        xi, yi = graph[i]
        for j in (i+1):Nv
            xj, yj = graph[j]

            dij = sqrt((xi - xj)^2. + (yi - yj)^2.)
            if dij <= 1.0
                edges[i,j] = true
            end
        end
    end
    return findall(edges)
end

function Ω(t::Float64)
    if 0 <= t <= 0.25
        return (Ω_max / 0.25) * t
    elseif 0.25 < t <= 0.69
        return Ω_max
    elseif 0.69 < t <= 1
        return - Ω_max * t / 0.31 + Ω_max * (1 + 0.69/0.31)
    end
end

function δ(t::Float64)
    slope = (δ_0 - δ_max)/(0.25 - 0.69)
    if 0 <= t <= 0.25
        return δ_0
    elseif 0.25 < t <= 0.69
        return t * slope + (δ_max - slope * 0.69)
    elseif 0.69 < t <= 1
        return δ_max
    end
end 

function hamiltonian(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2]
        repeat(Nv,u*P1,(l,m))
    end |> sum
    interaction_term - δ(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω(t)*sum(map(i->put(Nv,i=>X), 1:Nv))
end

function run_annealing(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])
    for t in 0:dt:1.0
        h = hamiltonian(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt * 100)
    end
    return psi_t
end

# open("task2_data.dat","w") do io
#     for sample in measure(psi; nshots=10_000)
#         println(io, sample)
#     end
# end

run_annealing (generic function with 1 method)

In [6]:
graph = [(0.3461717838632017, 1.4984640297338632), 
         (0.6316400411846113, 2.5754677320579895), 
         (1.3906262250927481, 2.164978861396621), 
         (0.66436005100802, 0.6717919819739032), 
         (0.8663329771713457, 3.3876341010035995), 
         (1.1643107343501296, 1.0823066243402013)
        ]
edges = get_edges(graph)
dt = 0.001
psi = run_annealing(graph, edges, dt)

ArrayReg{1, ComplexF64, Array...}
    active qubits: 6/6

In [7]:
get_edges(graph)

5-element Vector{CartesianIndex{2}}:
 CartesianIndex(2, 3)
 CartesianIndex(1, 4)
 CartesianIndex(2, 5)
 CartesianIndex(1, 6)
 CartesianIndex(4, 6)

In [9]:
samples = measure(psi; nshots = 10_000)
sort(freqtable(samples), rev = true)[1:5]

5-element Named Vector{Int64}
Dim1       │ 
───────────┼─────
011100 ₍₂₎ │ 2879
110100 ₍₂₎ │ 2839
010101 ₍₂₎ │ 2788
011101 ₍₂₎ │  313
110101 ₍₂₎ │  301

In [10]:
using PyCall
plt = pyimport("matplotlib.pyplot") 
nx  = pyimport("networkx")

PyObject <module 'networkx' from '/Users/Eden/.julia/conda/3/lib/python3.8/site-packages/networkx/__init__.py'>

In [14]:
fig, ax = plt.subplots()

G=nx.Graph()

for i in range(6)
    G.add_node(i,pos=(graph[i][0],graph[i][1]))
end

edge_list=[]
for i in range(6)
    for j in range(i+1,6)
        if edges[i,j]==True
            G.add_edge(i,j)
            edge_list.append([i,j])
            end
        end
    end
pos=nx.get_node_attributes(G,'pos')

nx.draw_networkx_nodes(G,pos, ax=ax, node_color='y')
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos, edgelist=edge_list, edge_color='y')

ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

# plt.xlim([0, 4.0])
# plt.xlim([0, 4.0])
# plt.show()

LoadError: ArgumentError: At least one of `length` or `stop` must be specified